# La patogénesis del alzhéimer: clasificación precoz mediante redes neuronales
_**Objetivo:** distinguir a sujetos que puedan desarrollar alzhéimer con una exactitud superior al 65% y una antelación al diagnóstico de más de tres años._

_**Datos:** del proyecto [OASIS 3](https://www.oasis-brains.org/)._

### Tarea previa: obtener el listado de sesiones
1. Pedir acceso en https://www.oasis-brains.org/#access
2. Esperar a la concesión
3. Crear un usuario mediante el mail de confirmación recibido
4. Acceder a la plataforma [XNAT](https://central.xnat.org/)
5. Identificarse
6. Abrir el proyecto OASIS3
7. Abrir la pestaña (tab) MR Sessions
8. Descargar fichero CSV en: Options > Spreadsheet
9. Colocar el fichero en el mismo directorio de este notebook

### Obtención de datos
Se recomienda el uso de scripts oficiales: `git clone https://github.com/NrgXnat/oasis-scripts.git`
> Son necesarios para la ejecución de este notebook. Se asume que se localizan su directorio madre.

In [1]:
import subprocess

subprocess.call('../oasis-scripts/download_scans/download_oasis_scans_')